In [1]:
import tensorflow as tf
import os
import cv2
import imghdr

C:\Users\vrajp\AppData\Local\Temp\ipykernel_20116\1916308919.py:4: DeprecationWarning: 'imghdr' is deprecated and slated for removal in Python 3.13
  import imghdr


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
data_dir = 'ayurveda_short/train'
val_dir = 'ayurveda_short/test'

In [4]:
img_exts = ['jpg', 'png', 'bmp', 'jpeg']

In [5]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in img_exts:
                print("Image not in ext list {}".format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("Issue with image {}".format(image_path))

In [6]:
for image_class in os.listdir(val_dir):
    for image in os.listdir(os.path.join(val_dir, image_class)):
        image_path = os.path.join(val_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in img_exts:
                print("Image not in ext list {}".format(image_path))
                os.remove(image_path)
        except Exception as e:
            print("Issue with image {}".format(image_path))

In [7]:
data = tf.keras.utils.image_dataset_from_directory('ayurveda_short/train')
val_data = tf.keras.utils.image_dataset_from_directory('ayurveda_short/test')


Found 4657 files belonging to 23 classes.
Found 1187 files belonging to 23 classes.


In [8]:
data = data.map(lambda x, y: (x / 255, y))
val_data = val_data.map(lambda x, y: (x / 255, y))


In [9]:
train_size = int(len(data) * 0.7)
test_size = int(len(data) * 0.3)+1
val_size = int(len(val_data))

In [10]:
train = data.take(train_size)
val = val_data.take(val_size)
test = data.skip(train_size + val_size).take(test_size)


In [11]:
num_classes = len(os.listdir(data_dir))


In [12]:
input_shape = (256, 256, 3)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), 1, activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(32, (3, 3), 1, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Conv2D(16, (3, 3), 1, activation='relu'),
    tf.keras.layers.MaxPooling2D(),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')  
])

In [14]:
model.compile(optimizer='adam',
              loss=tf.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [15]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 127, 127, 16)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 16)        4624      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 30, 30, 16)        0

In [17]:
history = model.fit(train, epochs=20, validation_data=val)

Epoch 1/20


102/102 [==============================] - 29s 278ms/step - loss: 0.3532 - accuracy: 0.9182 - val_loss: 7.5431 - val_accuracy: 0.1896
Epoch 2/20
102/102 [==============================] - 28s 275ms/step - loss: 0.2807 - accuracy: 0.9332 - val_loss: 8.6011 - val_accuracy: 0.1794
Epoch 3/20
102/102 [==============================] - 28s 274ms/step - loss: 0.2285 - accuracy: 0.9476 - val_loss: 9.3835 - val_accuracy: 0.1702
Epoch 4/20
102/102 [==============================] - 29s 278ms/step - loss: 0.1848 - accuracy: 0.9568 - val_loss: 10.1756 - val_accuracy: 0.1693
Epoch 5/20
102/102 [==============================] - 28s 274ms/step - loss: 0.1963 - accuracy: 0.9605 - val_loss: 9.8316 - val_accuracy: 0.1676
Epoch 6/20
102/102 [==============================] - 28s 275ms/step - loss: 0.1582 - accuracy: 0.9672 - val_loss: 10.3644 - val_accuracy: 0.1735
Epoch 7/20
102/102 [==============================] - 28s 276ms/step - loss: 0.1134 - accuracy: 0.9782 - val_loss: 10.6283 - val_accuracy: 

In [18]:
loss, accuracy = model.evaluate(test)
print("Test Accuracy:", accuracy)

6/6 [==============================] - 5s 59ms/step - loss: 10.6154 - accuracy: 0.1582
Test Accuracy: 0.15819208323955536


In [ ]:
import numpy as np

In [ ]:
img = cv2.imread('acne.jpeg')
resize = tf.image.resize(img, (256, 256))
resize = np.expand_dims(resize / 255, 0)
prediction = model.predict(resize)
predicted_class = np.argmax(prediction)
print("Predicted Class:", predicted_class)

In [ ]:

os.listdir('train')[predicted_class]